In [1]:
import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan


loan = pd.read_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\01_Loans\20220630\infocube_01_loan20220630.txt', sep = ",", header = 0, low_memory = False)


IF  = pd.read_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\2022\202206\working\Total_Impaired_Final_20220630.txt',sep = ",", header = 0, low_memory = False)


date = 20220630

location_cust = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\02_Teradata\01_Credit_Risk_Reporting\01_Cust\\"+str(date)+"\\S001_004_ACRM_CUST"+str(date)+".txt"
cust = pd.read_csv(location_cust, sep = "|", header = 0, error_bad_lines = False)
cust.columns = cust.columns.str.replace(" ", "_")
cust.M_SUB_SUB_MARKET_SEGMENT.replace({'    ': -9999}, inplace=True)

Total_Impaired  = pd.read_csv(r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\'+str(date)[:4]+'\\'+str(date)[:6]+'\working\Total_Impaired_Final_'+str(date)+'.txt',sep = ",", header = 0, low_memory = False)


In [28]:
loan.columns = loan.columns.str.upper()
loan.columns = loan.columns.str.strip()

IF.columns = IF.columns.str.upper()
IF.columns = IF.columns.str.strip()

loan.M_PRODUCT_HIERARCHY_CD = loan.M_PRODUCT_HIERARCHY_CD.str.strip()
loan.M_PRODUCT_HIERARCHY_CD = loan.M_PRODUCT_HIERARCHY_CD.str.upper()


loan1 = loan.iloc[np.where((loan.M_PRODUCT_HIERARCHY_CD.isin(['H80','S54','S57','S58','W34','W54']))&(loan.M_FULL_WRITEOFF_FLG=='N')&(loan.M_ACCT_STATUS_CD==1))] #&(loan.M_SUB_MARKET_SEGMENT_CD == 2)

loan1 = loan1[['AS_OF_DATE','M_ACCOUNT_NO','M_CUS_NO','LEVEL_6','M_SUB_MARKET_SEGMENT_CD','M_PRODUCT_HIERARCHY_CD','M_SOURCE_PROD_TYPE_CD','SOURCE_PROD_TYPE_DSC','M_MTHS_INSTALLMENT_IN_ARREARS','M_FINANCING_CONCEPT','M_BNM_BALANCE_SUM']]

#IF = IF[['V_ORIGINAL_ACCOUNT_NUMBER','MKT_SUB_SEGMENT','OUTSTANDING_AFTER_EIR_UWI','V_D_CUST_FIRST_NAME']]

In [39]:
loan.head(1)

,AS_OF_DATE,M_ACCOUNT_NO,M_CUS_NO,M_SOURCE_PROD_TYPE_CD,M_APPROVED_LIMIT_SUM,M_BNM_BALANCE_SUM,M_PRODUCT_HIERARCHY_CD,M_ECONOMIC_SECTOR_CD,BRANCH_CODE,M_DATE_AA_APPROVED,AMRT_TERM,M_DATE_FIRST_RELEASED,M_DATE_FINAL_RELEASED,M_NPL_DATE,M_MTHS_INSTALLMENT_IN_ARREARS,M_RESCHEDULE_FLG,M_RESTRUCTURE_FLG,M_INTEREST_INCOME_SUM,M_NON_INTEREST_INCOME_SUM,CUR_GROSS_RATE_SUM,M_PENALTY_CHARGES_SUM,M_MISC_CHARGES_SUM,M_FUNDS_CHARGE_MYR_SUM,M_INTEREST_INCOME_MYR_SUM,M_NET_INT_INCOME_MYR_SUM,M_SPECIAL_MENTION_ACCT_FLG,M_NEW_NPL_FLG,M_ENTITY_CD,M_ACCT_STATUS_CD,M_NPL_FLG,AVG_BOOK_BAL_SUM,ADJUSTABLE_TYPE_CD,MATURITY_DATE,M_FULL_WRITEOFF_FLG,M_INTEREST_RATE_TYPE,M_INTEREST_CODE,M_SUB_MARKET_SEGMENT_CD,M_AA_NUMBER,AMRT_TERM_MULT,M_FINANCING_CONCEPT,EAD_PROPORTIONAL_MYR,EXPECTED_LOSS_MYR,RWA_AFTER_RISK_MIT_MYR,APPROACH,SECTOR_CATEGORY,RSME_INCLUSION_INDICATOR,RS_INCLUSION_INDICATOR,REPORT_INDICATOR,PD_SEGMENT,LEVEL_6,LEVEL_5,ECON_SECTOR,REGION_DESC,STATE_DESC,BRANCH_DESC,CUSTOMER_SEGMENT,M_RACE_CD,SEX,M_NATIONALITY_CD,BTH_DT,M_OCC_SECTOR_CD,M_INCOME_GRP_CD,MARRIED,MBB_NATIONALITY_DESC,OCC_SECTOR,INCOME_GRP,M_SUB_MARKET_SEGMENT_DESC,RATING,RSME_RATING_GRADE,SOURCE_PROD_TYPE_DSC,INTEREST_RATE_DESC,NOB_CD,RACE,GENDER,BIRTHDAYYEAR,AGE,AGE_RANGE,MARITAL_STATUS,RELEASED_FLAG,ADJUSTABLE_TYPE_DESC,M_FUNDS_CHARGE_MYR,NOA
0,30-JUN-2022,464306121052,10000037,H4,541304.0,527280.79,H4,0324,6430,27-APR-2017,25.0,04-AUG-2017,06-MAR-2020,01-JAN-1900,0.0,N,N,1555.57,0.0,3.6,0.0,0.0,1050.11,1555.57,505.46,N,N,MIB,1.0,N,525777.07,250.0,06-MAR-2045,N,S,46,3.0,2017B13536,Y,B06,527280.79,1002.62,141017.26,6.0,9.0,0,M06,0.0,R3,Shophouse,Mortgage,Shophouses,SELANGOR/NS,SELANGOR,"JLN 222, PJ",3.0,3.0,2.0,1.0,2.05e+06,C10,13.0,1,NaN,NaN,NaN,CFS-IND,0.0,0,SHOPHOUSE FINANCING-I,NaN,NaN,Chinese,Male,1989.0,33.0,C. 30-34,Single,FULLY RELEASED,D.Other Adjustable,1050.11,1


In [38]:
cust.head(1)

,CUS_NO,NAME,CUSTOMER_SEGMENT,M_MARKET_SEGMENT_CD,M_SUB_MARKET_SEGMENT,M_SUB_SUB_MARKET_SEGMENT,M_CUSTOMER_TYPE,INDUSTRIAL_SECTOR
0,20715737,NUR FARAH HANUM BINTI SULAIMAN,004,1,3,-9999,101,97000


In [37]:
loan1.M_CUS_NO = loan1.M_CUS_NO.astype('str')
loan1.M_CUS_NO = loan1.M_CUS_NO.str.strip()
loan1.M_CUS_NO = loan1.M_CUS_NO.str.upper()

cust.CUS_NO = cust.CUS_NO.astype('str')
cust.CUS_NO = cust.CUS_NO.str.strip()
cust.CUS_NO = cust.CUS_NO.str.upper()

D:\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [40]:
join_b = loan1.merge(cust[['CUS_NO','NAME']].rename(columns={'CUS_NO':'M_CUS_NO'}), on='M_CUS_NO', how='left')

In [41]:
Total_Impaired['IF_tag'] = 'Y'

sumA = Total_Impaired.fillna(0).groupby(['V_ORIGINAL_ACCOUNT_NUMBER','V_D_CUST_FIRST_NAME','IF_tag','MKT_SUB_SEGMENT'])[['OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()

sumB = Total_Impaired.fillna(0).groupby(['CUSTOMER_ID','V_D_CUST_FIRST_NAME','IF_tag','MKT_SUB_SEGMENT'])[['OUTSTANDING_AFTER_EIR_UWI']].sum().reset_index()


In [42]:
sumA.V_ORIGINAL_ACCOUNT_NUMBER.dtypes, join_b.M_ACCOUNT_NO.dtypes

(dtype('O'), dtype('int64'))

In [43]:
sumB.CUSTOMER_ID.dtypes, join_b.M_CUS_NO.dtypes

(dtype('int64'), dtype('O'))

In [44]:
join_b.M_ACCOUNT_NO = join_b.M_ACCOUNT_NO.astype('Int64').astype('str') #leh guna skali je
sumB.CUSTOMER_ID = sumB.CUSTOMER_ID.astype('Int64').astype('str') #leh guna skali je

In [45]:
loan_combine = join_b.merge(sumA.drop_duplicates().rename(columns={'V_ORIGINAL_ACCOUNT_NUMBER':'M_ACCOUNT_NO'}),on='M_ACCOUNT_NO',how='left')

bygcif = join_b.merge(sumB.drop_duplicates().rename(columns={'CUSTOMER_ID':'M_CUS_NO'}),on='M_CUS_NO',how='left')

In [46]:
writer = pd.ExcelWriter(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\01_Reporting\03_Adhoc_Reporting\2022\0722_HYPR\LOAN_SJPP"+str(date)[:6]+"(python).xlsx", engine='xlsxwriter')


loan_combine.to_excel(writer, sheet_name='W Impaired by Account', index = False)

bygcif.to_excel(writer, sheet_name='W Impaired by gcif', index = False)


writer.save()